In [1]:
import pandas as pd
from datetime import datetime
import os
import torch
import pickle
import matplotlib.pyplot as plt
import sqlalchemy as sal
import numpy as np

In [2]:
import sqlalchemy as sal

endpoint = "capstone.clihskgj8i7s.us-west-2.rds.amazonaws.com"
user="group3"
db="db1"

# endpoint = "capstone.clihskgj8i7s.us-west-2.rds.amazonaws.com"
# user="group3"
# db="db1"
pw="3qLXGOaxMSoqYfj3yAOY"

engine = sal.create_engine('postgresql://%s:%s@%s/%s' % (user, pw, endpoint, db))

In [3]:
import pandas as pd

merged_df = pd.read_pickle('traffic_covid.pkl')

In [4]:
cases = '''select record_date,new_cases from traffic_covid_both'''
cases = engine.execute(cases).fetchall()

import datetime
import pandas as pd

fromtime = datetime.datetime.strptime('01-01-2020','%m-%d-%Y')
cases = pd.DataFrame(cases)
cases[0] = pd.to_datetime(cases[0])
cases.set_index(0,inplace = True)
cases.fillna(0,inplace = True)
cases = cases[cases.index <= '2020-06-29']
covid = cases.fillna(0.).to_numpy().astype('float32') # This step was just for COVID encoder

In [5]:
from datetime import datetime

def convert_to_date(ind):
    return datetime.strptime(str(merged_df.index[ind].date()), '%Y-%m-%d')

def return_case_index(ind):
    #print(convert_to_date(ind))
    return cases.index.get_loc(convert_to_date(ind))

def relu(arr):
    arr[arr < 0] = 0.
    return arr

In [6]:
df = merged_df.drop('covid_cases',axis = 1)

# Fill in missing values

horizon = 12
rolling_data = df.rolling(horizon,min_periods=1).mean()
rolling_data.fillna(rolling_data.mean(),inplace = True)
df = df.fillna(rolling_data[df.isnull()])

num_samples, num_nodes = (merged_df.shape[0],merged_df.shape[1] - 1)

add_time_in_day = True
add_day_in_week = False

num_samples, num_nodes = df.shape
data = np.expand_dims(df.values, axis=-1)
data_list = [data]

if add_time_in_day:
    time_ind = (df.index.values - df.index.values.astype("datetime64[D]")) / np.timedelta64(1, "D")
    time_in_day = np.tile(time_ind, [1, num_nodes, 1]).transpose((2, 1, 0))
    data_list.append(time_in_day)
if add_day_in_week:
    day_in_week = np.zeros(shape=(num_samples, num_nodes, 7))
    day_in_week[np.arange(num_samples), :, df.index.dayofweek] = 1
    data_list.append(day_in_week)

data = np.concatenate(data_list, axis=-1)

In [7]:
data.shape

(52114, 320, 2)

In [8]:
x_offsets = np.sort(np.concatenate((np.arange(-11, 1, 1),)))
y_offsets = np.sort(np.arange(1, 13, 1))

min_t = abs(min(x_offsets))
max_t = abs(num_samples - abs(max(y_offsets)))

cov_horizon = 7
x = []
y = []
cov_x = []
cov_y = []

for t in range(min_t, max_t):
    x_t = data[t + x_offsets, ...]
    y_t = data[t + y_offsets, ...]
    cind = return_case_index(t)
    cov_x.append(cases.values[relu(np.arange(cind-cov_horizon,cind))])
    cov_y.append(cases.values[cind])
    x.append(x_t)
    y.append(y_t)
    
x = np.stack(x, axis=0)
y = np.stack(y, axis=0)

covid_cases = np.stack(cov_x,axis = 0).astype('float32')
#covid_labels = np.stack(cov_y,axis = 0).astype('float32')

b = np.transpose(np.tile(covid_cases,(2,1,1,num_nodes)),(2,1,3,0))
a = np.transpose(x,(1,0,2,3))

x = np.transpose(np.vstack([a,b]),(1,0,2,3))

In [9]:
!mkdir -p data_test

In [10]:
num_samples = x.shape[0]
num_test = round(num_samples * 0.2)
num_train = round(num_samples * 0.7)
num_val = num_samples - num_test - num_train

x_train, y_train = x[:num_train], y[:num_train]

# val
x_val, y_val = (
    x[num_train: num_train + num_val],
    y[num_train: num_train + num_val],
)

# test
x_test, y_test = x[-num_test:], y[-num_test:]

for cat in ["train", "val", "test"]:
    _x, _y = locals()["x_" + cat], locals()["y_" + cat]
    print(cat, "x: ", _x.shape, "y:", _y.shape)
    np.savez_compressed(
        os.path.join('data_test', "%s.npz" % cat),
        x=_x,
        y=_y,
        x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
        y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
    )

train x:  (36464, 19, 320, 2) y: (36464, 12, 320, 2)
val x:  (5209, 19, 320, 2) y: (5209, 12, 320, 2)
test x:  (10418, 19, 320, 2) y: (10418, 12, 320, 2)


In [9]:
from torchts.utils import data

In [ ]:
dataset = data.load_dataset('data_test',64)